<a href="https://colab.research.google.com/github/PrettyCharity/Bigquery/blob/main/Stackoverflow_Bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Google Cloud Credentials
import os
from google.cloud import bigquery
# Credentials and the path
credential_path = "key.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

In [2]:
#@title Accessing the 'stackoverflow' via bigquery object
client = bigquery.Client()
dataset_ref = client.dataset('stackoverflow', project = 'bigquery-public-data')
dataset = client.get_dataset(dataset_ref)

In [3]:
#@title Exploring the tables of stackoverflow
tables = client.list_tables(dataset)
for table in tables:
  print(table.table_id)

badges
comments
post_history
post_links
posts_answers
posts_moderator_nomination
posts_orphaned_tag_wiki
posts_privilege_wiki
posts_questions
posts_tag_wiki
posts_tag_wiki_excerpt
posts_wiki_placeholder
stackoverflow_posts
tags
users
votes


In [4]:
#@title Obtaining 'posts_questions' table
table_ref = dataset_ref.table('posts_questions')
table = client.get_table(table_ref)
preview = client.list_rows(table, max_results = 5).to_dataframe()
print('Table posts_questions columns:\n\n')
print(preview.columns)

Table posts_questions columns:


Index(['id', 'title', 'body', 'accepted_answer_id', 'answer_count',
       'comment_count', 'community_owned_date', 'creation_date',
       'favorite_count', 'last_activity_date', 'last_edit_date',
       'last_editor_display_name', 'last_editor_user_id', 'owner_display_name',
       'owner_user_id', 'parent_id', 'post_type_id', 'score', 'tags',
       'view_count'],
      dtype='object')


In [5]:
#@title Obtaining 'posts_answers' table
table_ref = dataset_ref.table('posts_answers')
table = client.get_table(table_ref)
preview = client.list_rows(table, max_results = 5).to_dataframe()
print('Table posts_answers columns:\n\n')
print(preview.columns)

Table posts_answers columns:


Index(['id', 'title', 'body', 'accepted_answer_id', 'answer_count',
       'comment_count', 'community_owned_date', 'creation_date',
       'favorite_count', 'last_activity_date', 'last_edit_date',
       'last_editor_display_name', 'last_editor_user_id', 'owner_display_name',
       'owner_user_id', 'parent_id', 'post_type_id', 'score', 'tags',
       'view_count'],
      dtype='object')


In [7]:
#@title Construct the SQL query to find bigquery experts of SOF
query = """
            SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
            FROM `bigquery-public-data.stackoverflow.posts_questions` as q
            INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` as a
                  ON q.id = a.parent_id
            WHERE q.tags LIKE '%bigquery%'
            GROUP BY a.owner_user_id
            ORDER BY number_of_answers DESC
        """ 


In [11]:
#@title Fetch data from the server and show the results
# API request - run the query, and convert the results to a pandas DataFrame
query_job = client.query(query)
bigquery_experts = query_job.to_dataframe()

bigquery_experts.head(10)

,user_id,number_of_answers
0,5221944.0,4497
1,1144035.0,1636
2,132438.0,900
3,6253347.0,736
4,1366527.0,620
5,243782.0,595
6,13473525.0,380
7,4490873.0,275
8,2877278.0,261
9,11206202.0,258
